[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/liulinbo19/MapLight-TDC/blob/main/tt_rdkit.ipynb)

In [1]:
!curl -o maplight.py https://raw.githubusercontent.com/liulinbo19/MapLight-TDC/main/tt_rdkit.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8658  100  8658    0     0  25434      0 --:--:-- --:--:-- --:--:-- 25464


In [2]:
# installs for Colab
!pip install rdkit PyTDC tqdm catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.3/151.3 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 83.2 MB/s eta 0:00

In [ ]:
from maplight import *
from tqdm import tqdm
import catboost as cb

from tdc.benchmark_group import admet_group

In [ ]:
benchmark_config = {
    'caco2_wang': ('regression', False),
    'bioavailability_ma': ('binary', False),
    'lipophilicity_astrazeneca': ('regression', False),
    'solubility_aqsoldb': ('regression', False),
    'hia_hou': ('binary', False),
    'pgp_broccatelli': ('binary', False),
    'bbb_martins': ('binary', False),
    'ppbr_az': ('regression', False),
    'vdss_lombardo': ('regression', True),
    'cyp2c9_veith': ('binary', False),
    'cyp2d6_veith': ('binary', False),
    'cyp3a4_veith': ('binary', False),
    'cyp2c9_substrate_carbonmangels': ('binary', False),
    'cyp2d6_substrate_carbonmangels': ('binary', False),
    'cyp3a4_substrate_carbonmangels': ('binary', False),
    'half_life_obach': ('regression', True),
    'clearance_hepatocyte_az': ('regression', True),
    'clearance_microsome_az': ('regression', True),
    'ld50_zhu': ('regression', False),
    'herg': ('binary', False),
    'ames': ('binary', False),
    'dili': ('binary', False)
}

In [ ]:
group = admet_group(path = 'data/')

# 打开一个文件用于写入结果
with open('/content/tt_rdkit.txt', 'w') as f:
    # change comment to run all benchmarks
    for admet_benchmark in benchmark_config.keys():
    # for admet_benchmark in [list(benchmark_config.keys())[7]]:
        predictions_list = []
        for seed in tqdm([1, 2, 3, 4, 5]):
            benchmark = group.get(admet_benchmark)
            predictions = {}
            name = benchmark['name']
            train, test = benchmark['train_val'], benchmark['test']

            # --------------------------------------------- #
            #  Train your model using train, valid, test    #
            #  Save test prediction in y_pred_test variable #
            X_train = get_fingerprints(train['Drug'])
            X_test = get_fingerprints(test['Drug'])

            task, log_scale = benchmark_config[name]
            params = {
                    'random_strength': 2,
                    'random_seed': seed,
                    'verbose': 0,
                }

            if task == 'regression':
                Y_scaler = scaler(log=log_scale)
                Y_scaler.fit(train['Y'].values)
                train['Y_scale'] = Y_scaler.transform(train['Y'].values)

                params['loss_function'] = 'MAE'
                model = cb.CatBoostRegressor(**params)
                model.fit(X_train, train['Y_scale'].values)

                y_pred_test = Y_scaler.inverse_transform(model.predict(X_test)).reshape(-1)
            elif task == 'binary':
                params['loss_function'] = 'Logloss'
                model = cb.CatBoostClassifier(**params)
                model.fit(X_train, train['Y'].values)

                y_pred_test = model.predict_proba(X_test)[:, 1]
            # --------------------------------------------- #

            predictions[name] = y_pred_test
            predictions_list.append(predictions)

        results = group.evaluate_many(predictions_list)

        # 将结果写入文件
        f.write('\n\n{}'.format(results))
        print('\n\n{}'.format(results))